In [1]:
"""

To recreate the same results with the pseudo-random numbers simply run all the cells once in order from top to bottom

"""

import numpy as np
import matplotlib.pyplot as plt
import random
from scipy.stats import norm

seed = 7654
np.random.seed(seed)

# defining all the constants

pi = np.pi

qed_coupling = 1/129

qcd_coupling = 0.118

zboson_mass = 91.2

zboson_decaywidth = 2.5

weinberg_angle = 0.223

electron_charge = -1

upquark_charge = 2/3

downquark_charge = -1/3

upquark_isospin = 1/2

downquark_isospin = -1/2

number_of_quark_flavours = 5

number_of_qcd_colours = 3

conversion_factor = 3.89379656*10**8

kappa = 1/(4*weinberg_angle*(1-weinberg_angle))

electron_vectorcoupling = downquark_isospin - 2* electron_charge*weinberg_angle

upquark_vectorcoupling = upquark_isospin - 2* upquark_charge*weinberg_angle

downquark_vectorcoupling= downquark_isospin - 2* downquark_charge*weinberg_angle

In [2]:
def chi_1(s): 

    return kappa* (s*(s- zboson_mass**2))/((s- zboson_mass**2)**2 + zboson_decaywidth**2 * zboson_mass**2)

def chi_2(s):

    return kappa**2 * (s**2)/((s- zboson_mass**2)**2 + zboson_decaywidth**2 * zboson_mass**2)
    
#defining the matrix element e^+ e^- --> qq
#here with the explicit summing over the flavours

def matrix_element(s,cos_theta,quark_flavour):

    if (quark_flavour == 2 or quark_flavour ==5): 
        return (4*pi*qed_coupling)**2 * number_of_qcd_colours* ((1+ cos_theta**2)*(electron_charge**2*upquark_charge**2 + 2* electron_charge * upquark_charge* electron_vectorcoupling * upquark_vectorcoupling * chi_1(s) + (downquark_isospin**2+ electron_vectorcoupling**2)*(upquark_isospin**2 + upquark_vectorcoupling**2)*chi_2(s))+ cos_theta* (4* electron_charge*upquark_charge*downquark_isospin*upquark_isospin*chi_1(s)+ 8* downquark_isospin*electron_vectorcoupling* upquark_isospin* upquark_vectorcoupling*chi_2(s)))
    
    else:  
        return (4*pi*qed_coupling)**2 * number_of_qcd_colours* ((1+ cos_theta**2)*(electron_charge**2*downquark_charge**2 + 2* electron_charge * downquark_charge* electron_vectorcoupling * downquark_vectorcoupling * chi_1(s) + (downquark_isospin**2+ electron_vectorcoupling**2)*(downquark_isospin**2 + downquark_vectorcoupling**2)*chi_2(s))+ cos_theta* (4* electron_charge*downquark_charge*downquark_isospin*downquark_isospin*chi_1(s)+ 8* downquark_isospin*electron_vectorcoupling* downquark_isospin* downquark_vectorcoupling*chi_2(s)))

def differential_cross_section(s,cos_theta):

    summed_matrix_elements = 0
    
    for i in range(number_of_quark_flavours):
    
        quark_flavour = i+1
        
        summed_matrix_elements = summed_matrix_elements + matrix_element(s,cos_theta,quark_flavour)
    
    return conversion_factor * 1/(8*pi) *1/(4*pi) * 1/(2*s) *summed_matrix_elements

In [3]:
#monte carlo integrator
def mc_integrator(function, low_limits, up_limits, number_of_samples):

    dimension = len(low_limits)
    
    integration_volume = np.prod(up_limits- low_limits)
    
    samples = np.random.uniform(low_limits, up_limits, size=(number_of_samples, dimension))
    
    function_values = function(zboson_mass**2,samples[:,0])
    
    integral = integration_volume * np.mean(function_values)
    
    error = integration_volume * np.std(function_values) / np.sqrt(number_of_samples)
    
    return integral,error

In [4]:
#doing the integration with explicit summing over the flavours
low_limits = np.array([-1,0])

up_limits = np.array([1,2*pi])

integration_value_1a  = [mc_integrator(differential_cross_section,low_limits,up_limits,20000) for _ in range(10000)]

with open('../Data_and_Plots/data1.1aPart1.txt', 'wb') as f:
    header = 'MCestimate  MCerror\n'
    header_ascii = header.encode('ascii')
    f.write(header_ascii)
    for i in integration_value_1a:
        line = str(i[0])+ ' ' + str(i[1]) + '\n'
        line_ascii = line.encode('ascii')
        f.write(line_ascii)